In [1]:
import cv2
import itertools as it
import numpy as np
import random
import threading
from captcha.image import ImageCaptcha
from utilities import *

In [2]:
labels = [f'{chars:>04}' for chars in range(10000)]
random.shuffle(labels)

In [3]:
with Timer('one thread'):
    l = [ImageCaptcha().generate_image(l) for l in labels]
width, height = l[0].width, l[0].height
len(l), width, height

one thread time: 24.232515 seconds


(10000, 160, 60)

In [4]:
n = 10
l = n * [None]
def fn(k):
    l[k] = [ImageCaptcha().generate_image(labels[i]) for i in range(1000 * k, 1000 * (k + 1))]
threads = [threading.Thread(target=fn, args=(i,)) for i in range(n)]
for thread in threads:
    thread.start()
with Timer('multiple threads'):
    for thread in threads:
        thread.join()
l = list(it.chain.from_iterable(l))
width, height = l[0].width, l[0].height
len(l), width, height

multiple threads time: 24.746306 seconds


(10000, 160, 60)

In [5]:
image = np.array(l[0].getdata()).reshape((height, width, 3)).astype(np.uint8)
cv2.imshow('tesst', image)
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
with Timer('one thread'):
    ll = [np.array(v.getdata()) for v in l]
len(ll), ll[0].shape, ll[0].dtype

one thread time: 40.337646 seconds


(10000, (9600, 3), dtype('int32'))

In [7]:
n = 10
ll = n * [None]
def fn(k):
    ll[k] = [np.array(l[i].getdata()) for i in range(1000 * k, 1000 * (k + 1))]
threads = [threading.Thread(target=fn, args=(i,)) for i in range(n)]
for thread in threads:
    thread.start()
with Timer('multiple threads'):
    for thread in threads:
        thread.join()
ll = list(it.chain.from_iterable(ll))
len(ll), ll[0].shape, ll[0].dtype

multiple threads time: 36.896506 seconds


(10000, (9600, 3), dtype('int32'))

In [8]:
n=9
image = np.vstack(ll).reshape((len(ll),60,160,3)).astype(np.uint8)
cv2.imshow('tesst', image[n])
cv2.waitKey()
cv2.destroyAllWindows()
image.shape, labels[n]

((10000, 60, 160, 3), '3645')

In [9]:
n=9
image = np.vstack(ll).reshape((len(ll)*60,160,3)).astype(np.uint8)
image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY).reshape((len(ll),60,160))
cv2.imshow('tesst', image[n])
cv2.waitKey()
cv2.destroyAllWindows()
image.shape, labels[n]

((10000, 60, 160), '3645')

In [10]:
n=9
image = np.vstack(ll[:n]).reshape((n*60,160,3)).astype(np.uint8)
cv2.imshow('tesst', image)
cv2.waitKey()
cv2.destroyAllWindows()
image.shape

(540, 160, 3)